In [ ]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [ ]:
multiply.invoke({"a": 6, "b": 7})  # returns 42

In [ ]:
tool_call = {"type": "tool_call", "id": "1", "args": {"a": 42, "b": 7}}
multiply.invoke(tool_call)  # returns a ToolMessage object

In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


agent = create_react_agent(model="anthropic:claude-3-7-sonnet", tools=[multiply])
agent.invoke({"messages": [{"role": "user", "content": "what's 42 x 7?"}]})

In [ ]:
from dataclasses import dataclass
from typing import Literal

from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.runtime import Runtime


@dataclass
class CustomContext:
    tools: list[Literal["weather", "compass"]]


@tool
def weather() -> str:
    """Returns the current weather conditions."""
    return "It's nice and sunny."


@tool
def compass() -> str:
    """Returns the direction the user is facing."""
    return "North"


model = init_chat_model("anthropic:claude-sonnet-4-20250514")


def configure_model(state: AgentState, runtime: Runtime[CustomContext]):
    """Configure the model with tools based on runtime context."""
    selected_tools = [tool for tool in [weather, compass] if tool.name in runtime.context.tools]
    return model.bind_tools(selected_tools)


agent = create_react_agent(
    # Dynamically configure the model with tools based on runtime context
    configure_model,
    # Initialize with all tools available
    tools=[weather, compass],
)

output = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Who are you and what tools do you have access to?",
            }
        ]
    },
    context=CustomContext(tools=["weather"]),  # Only enable the weather tool
)

print(output["messages"][-1].text())

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="claude-3-5-haiku-latest")

model_with_tools = model.bind_tools([multiply])

In [ ]:
from langchain_core.messages import AIMessage
from langgraph.prebuilt import ToolNode

# Define tools


def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    else:
        return "It's 90 degrees and sunny."


def get_coolest_cities():
    """Get a list of coolest cities"""
    return "nyc, sf"


tool_node = ToolNode([get_weather, get_coolest_cities])

message_with_multiple_tool_calls = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_coolest_cities",
            "args": {},
            "id": "tool_call_id_1",
            "type": "tool_call",
        },
        {
            "name": "get_weather",
            "args": {"location": "sf"},
            "id": "tool_call_id_2",
            "type": "tool_call",
        },
    ],
)

tool_node.invoke({"messages": [message_with_multiple_tool_calls]})

In [ ]:
from langchain_core.tools import tool


@tool("multiply_tool", parse_docstring=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers.

    Args:
        a: First operand
        b: Second operand
    """
    return a * b

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool


class MultiplyInputSchema(BaseModel):
    """Multiply two numbers"""

    a: int = Field(description="First operand")
    b: int = Field(description="Second operand")


@tool("multiply_tool", args_schema=MultiplyInputSchema)
def multiply(a: int, b: int) -> int:
    return a * b

In [ ]:
from langchain_core.tools import tool


@tool("multiply_tool")
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [ ]:
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig


@tool
def get_user_info(config: RunnableConfig) -> str:
    """Retrieve user information based on user ID."""
    user_id = config["configurable"].get("user_id")
    return "User is John Smith" if user_id == "user_123" else "Unknown user"


# Invocation example with an agent
agent.invoke(
    {"messages": [{"role": "user", "content": "look up user info"}]},
    config={"configurable": {"user_id": "user_123"}},
)

In [ ]:
from typing import Annotated, NotRequired
from langchain_core.tools import tool
from langgraph.prebuilt import InjectedState, create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState


class CustomState(AgentState):
    # The user_name field in short-term state
    user_name: NotRequired[str]


@tool
def get_user_name(state: Annotated[CustomState, InjectedState]) -> str:
    """Retrieve the current user-name from state."""
    # Return stored name or a default if not set
    return state.get("user_name", "Unknown user")


# Example agent setup
agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_user_name],
    state_schema=CustomState,
)

# Invocation: reads the name from state (initially empty)
agent.invoke({"messages": "what's my name?"})

In [ ]:
from typing import Annotated
from langgraph.types import Command
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool, InjectedToolCallId


@tool
def update_user_name(new_name: str, tool_call_id: Annotated[str, InjectedToolCallId]) -> Command:
    """Update user-name in short-term memory."""
    return Command(
        update={
            "user_name": new_name,
            "messages": [
                ToolMessage(f"Updated user name to {new_name}", tool_call_id=tool_call_id)
            ],
        }
    )

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langgraph.graph import StateGraph
from langgraph.config import get_store


@tool
def get_user_info(config: RunnableConfig) -> str:
    """Look up user info."""
    # Same as that provided to `builder.compile(store=store)`
    # or `create_react_agent`
    store = get_store()
    user_id = config["configurable"].get("user_id")
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"


builder = StateGraph(...)
...
graph = builder.compile(store=store)

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langgraph.graph import StateGraph
from langgraph.config import get_store


@tool
def save_user_info(user_info: str, config: RunnableConfig) -> str:
    """Save user info."""
    # Same as that provided to `builder.compile(store=store)`
    # or `create_react_agent`
    store = get_store()
    user_id = config["configurable"].get("user_id")
    store.put(("users",), user_id, user_info)
    return "Successfully saved user info."


builder = StateGraph(...)
...
graph = builder.compile(store=store)

In [ ]:
@tool(return_direct=True)
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

In [ ]:
@tool(return_direct=True)
def greet(user_name: str) -> int:
    """Greet user."""
    return f"Hello {user_name}!"


tools = [greet]

configured_model = model.bind_tools(
    tools,
    # Force the use of the 'greet' tool
    tool_choice={"type": "tool", "name": "greet"},
)

In [ ]:
model.bind_tools(tools, parallel_tool_calls=False)

In [ ]:
from langchain_core.messages import AIMessage
from langgraph.prebuilt import ToolNode


def multiply(a: int, b: int) -> int:
    if a == 42:
        raise ValueError("The ultimate error")
    return a * b


# Default error handling (enabled by default)
tool_node = ToolNode([multiply])

message = AIMessage(
    content="",
    tool_calls=[
        {"name": "multiply", "args": {"a": 42, "b": 7}, "id": "tool_call_id", "type": "tool_call"}
    ],
)

result = tool_node.invoke({"messages": [message]})

In [ ]:
tool_node = ToolNode([multiply], handle_tool_errors=False)

In [ ]:
tool_node = ToolNode(
    [multiply], handle_tool_errors="Can't use 42 as the first operand, please switch operands!"
)

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(model="anthropic:claude-3-7-sonnet-latest", tools=[multiply])

# Default error handling
agent.invoke({"messages": [{"role": "user", "content": "what's 42 x 7?"}]})

In [ ]:
custom_tool_node = ToolNode([multiply], handle_tool_errors="Cannot use 42 as a first operand!")

agent_custom = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest", tools=custom_tool_node
)

agent_custom.invoke({"messages": [{"role": "user", "content": "what's 42 x 7?"}]})

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(model="openai:gpt-4o-mini", tools=[{"type": "web_search_preview"}])
response = agent.invoke({"messages": ["What was a positive news story from today?"]})